# Image Scaling with Machine Learning
## Inputs:
    1) Number of image pairs
    2) dictionary of {size_label_A: [pixels_high, pixels_wide], 
                      size_label_B: [pixels_high, pixels_wide]}
    3) run directory
    4) output directory
    5) existing sets parameters.json (None for first run)
***
## Parameters to generate a set of images for train-test-validate
    * Over all parameters
        * number of image sets
        * sizes dict {name1: [h, w], name2: [h, w],...}
        * run-limits {escape_boundry: diagonal_multiplier, max_iterations: int}
        * coloring scheme - perhaps a function
        * output file format
        * ouput file naming format
    * File system
        * output directory {train: path, test: path, validate: path}
        * run-log location
    * Stocastic Choices:
        * hashed dictionary - to reject duplicate images
        * equation
            * parameters
        * rotation 
        * center point 
        * frame scale

## Source structure
    * main - opens run_parameters file - passes to module function
    * selection and tracking module
    * starting plain complex
    * equations module
    * iteration module
    * matrix to image module
***
[PIL image](https://pillow.readthedocs.io/en/5.1.x/reference/Image.html#the-image-class) <br>
****
[GitHub scalygraphic](https://github.com/dlanier/scalygraphic/) <br>
### Background Code:
[Add: random plane on domain](https://github.com/dlanier/scalygraphic/blob/master/scalygraphic/zplain.py) <br>

#### Import & Expand *functions_demo.py*
[FMF functions_demo](https://github.com/dlanier/FlyingMachineFractal/blob/master/pyreimpic/functions_demo_01.py) <br>
#### Import & Trim itergators.py
[FMF itergators](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/itergataters.py) <br>
#### Rewrite HSV coloring
[FMF graphic_utility.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/graphic_utility.py) <br>
[FMF numcolorpy.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/numcolorpy.py) <br>
#### Reslove plain functions
[FMF z_plane.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/z_plane.py) <br>
****
    
### Inception: Random Selection, Tracking and Generation - called from *main()*

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time

sys.path.insert(0, '../src/')
sys.path.insert(0, 'scalygraphic/src/')

# import zplain as zp
# import eq_iter
# import deg_0_ddeq
# import numcolorpy as ncp
from im_scale_products import *
import impute_color as ncp

# Clickety Clack:
## production_parameters: *output_dir_name, image_sizes_dict, number_of_sets*
### get a random eq & p with a random domain
#### check uniquness
### get a random color map
## Iterate: run, write, report

In [ ]:
# %load ../src/im_scale_products.py

# Demos:

****
## get a random function and parameter set

In [ ]:
n_trys = 5

for k in range(n_trys):
    fcn_name, eq, p = get_rand_eq_p_set()
    
    print('\n\t', fcn_name)
    try:
        Z = 0.0+0.0j
        print(Z)
        print(eq(Z,p))
        Z = 1.0+1.0j
        print(Z)
        print(eq(Z,p))
    except:
        print('Crash crash ')
        break
        pass

## get a random complex domain

In [ ]:
N_DEC=4

domain_dict = get_random_domain()
f = zp.get_frame_from_dict(domain_dict)
s = zp.complex_frame_dict_to_string(f, N_DEC)
print(s,'\n%50s'%('(from dictionary)'))
for k, v in domain_dict.items():
    if isinstance(v, complex):
        v_str = zp.complex_to_string(v)
    else:
        v_str = '%i'%(v)
        
    print('%40s: %s'%(k, v_str))

## *hash_parameters( )* - function, parameters & domain - (reject duplicate parameters)
    Option: hashed dict > json file to allow reproduction of an exact data-set with different scaling

In [ ]:
fcn_name, eq, p = get_rand_eq_p_set()
domain_dict = get_random_domain()

hash_key = hash_parameters(domain_dict, fcn_name, p)
print(hash_key)

## Produce many pair of scaled images

In [ ]:
#             or use saved list
hash_list=[]

In [ ]:
cell_start_time = time.time()

n_2_do = 2
iteration_dict = {'it_max': 64, 'scale_dist': 10}
small_scale = [128, 128]
large_scale = [255, 255]

output_directory = '../../ImagesFriday'
if os.path.isdir(output_directory) == False:
    os.makedirs(output_directory)

hash_list = write_n_image_sets(n_2_do, 
                               iteration_dict['it_max'], 
                               iteration_dict['scale_dist'], 
                               small_scale, 
                               large_scale, 
                               output_directory, 
                               hash_list)

tt = time.time() - cell_start_time
print('\ntotal cell time: %0.2f seconds'%(tt))

## file nameing ?

In [ ]:
do_da = now_name(prefi_str=fcn_name, suffi_str='.jpg')
print(do_da)
d0_dat = hash_list[0] + '.jpg'
print(d0_dat)

## coloring logic
    * HSV model - choose three result components
        * Hue        (red to red) (0, 1) * 255          == visible spectrum colors
        * Saturation (gray to solid color) (0, 1) * 255 == color intensity
        * Value      (black to white) (0, 1) * 255      == brightness
    * RGB conversion from HSV
        * portable for multiple file formats
        * norm for train - validate - test
    * Escape-time algorithm produces
        * Escape Time (integer matrix)
        * Final Vector (Z - Z0)
            * distance (float matrix)
            * rotation (float matrix)
****
### Coloring component normalization:


In [ ]:
def view_component_color_range(Z0, Z, ET):
    # Examine color ranges - for HSV - RGB conversion 
    Zd_t, Zr_t, ETn_t = ncp.etg_norm(Z0, Z, ET)
    print('\nZd_t', np.max(Zd_t), np.min(Zd_t))
    print('Zr_t', np.max(Zr_t), np.min(Zr_t))
    print('ETn_t', np.max(ETn_t), np.min(ETn_t))

## Development legacy - cell below replaced by function:
```python
# Use hash_list for multi-session calling to avoid (improbable) duplicate parameter sets
hash_list = write_n_image_sets(n_2_do, iteration_dict, small_scale, large_scale, output_directory, hash_list)
```

In [ ]:
hashy_list = []

In [ ]:
cell_start_time = time.time()

n_2_do = 3

test_temporary_dir = '../../test_temporary_dir'
if os.path.isdir(test_temporary_dir) == False:
    os.makedirs(test_temporary_dir)

DISPLAY_IN_NOTEBOOK = True
GRAYSCALE = True

small_scale = [128, 128]
large_scale = [255, 255]
output_directory = '../../ImagesFriday'
if os.path.isdir(output_directory) == False:
    os.makedirs(output_directory)
    

for k_do in range(n_2_do):
    fcn_name, eq, p = get_rand_eq_p_set()
    print('\n%3i) %s'%(k_do+1, fcn_name))
    domain_dict = get_random_domain()

    domain_dict['it_max'] = 64
    domain_dict['max_d'] = 10 / domain_dict['zoom']

    hash_idx = hash_parameters(domain_dict, fcn_name, p)
    if hash_idx in hashy_list:
        print('\n\n\t\tImpossible! But! Skipping:\n%s\n'%(hash_idx))
    else:
        hashy_list.append(hash_idx)
        print(hash_idx + '\n')
        domain_dict['n_rows'] = small_scale[0]
        domain_dict['n_cols'] = small_scale[1]

        domain_dict['dir_path'] = test_temporary_dir

        list_tuple = [(eq, (p))]

        t0 = time.time()
        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        if GRAYSCALE == True:
            I = ncp.get_gray_im(ET, Z, Z0)
        else:
            I = ncp.get_im(ET, Z, Z0)
            
        file_name = os.path.join(output_directory, hash_idx + 'small.jpg')
        I.save(file_name)
        if DISPLAY_IN_NOTEBOOK:
            display(I)

        domain_dict['n_rows'] = large_scale[0]
        domain_dict['n_cols'] = large_scale[1]

        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        if GRAYSCALE == True:
            I = ncp.get_gray_im(ET, Z, Z0)
        else:
            I = ncp.get_im(ET, Z, Z0)
            
        file_name = os.path.join(output_directory, hash_idx + 'large.jpg')
        I.save(file_name)
        
        print('%0.3f\t 2 images time'%(time.time() - t0))
        if DISPLAY_IN_NOTEBOOK:
            display(I)

tt = time.time() - cell_start_time
print('\n%i pairs written in %0.2f seconds'%(n_2_do, tt))

## coloring logic
    * HSV model - choose three result components
        * Hue        (red to red) (0, 1) * 255          == visible spectrum colors
        * Saturation (gray to solid color) (0, 1) * 255 == color intensity
        * Value      (black to white) (0, 1) * 255      == brightness
    * RGB conversion from HSV
        * portable for multiple file formats
        * norm for train - validate - test
    * Escape-time algorithm produces
        * Escape Time (integer matrix)
        * Final Vector (Z - Z0)
            * distance (float matrix)
            * rotation (float matrix)
****
### Coloring component normalization:


In [ ]:
def view_component_color_range(Z0, Z, ET):
    # Examine color ranges - for HSV - RGB conversion 
    Zd_t, Zr_t, ETn_t = ncp.etg_norm(Z0, Z, ET)
    print('\nZd_t', np.max(Zd_t), np.min(Zd_t))
    print('Zr_t', np.max(Zr_t), np.min(Zr_t))
    print('ETn_t', np.max(ETn_t), np.min(ETn_t))
    
view_component_color_range(Z0, Z, ET)

## file nameing calls - time-stamp-name or hash-name

In [ ]:
do_da = now_name(prefi_str=fcn_name, suffi_str='.jpg')
print(do_da)
d0_dat = hashy_list[0] + '.jpg'
print(d0_dat)